# Get all US Congressional District information

## FIX ME
All of my data is linked by the FIPS identifying code for each county.
We will be using the FIPS data for plotting and id of county bounds and as a defacto index across all datasets.

We will also be using the county name for additional identification and the census tract area for each county. The census area is the surveyed area for census purposes (where people live) within the bounds of the county.

In [2]:

import os
from myfunctions import *
import requests

In [3]:
# # ONLY RUN THIS ONCE!!!!

# shape_url = 'https://theunitedstates.io/districts/cds/2016/'

# ## NY-30/shape.geojson

# states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
#           "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
#           "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
#           "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
#           "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]


# # for state in states:
# #     page = requests.get(URL)

# #     shape_urls = [x for x in range(1, 53)]

# all_shape_urls = []

# for state in states:
#     flag = 0
#     print(state)
#     for i in range(0, 54):
#         url = shape_url + state + '-' + str(i) + '/shape.geojson'
#         r = requests.get(url)
#         if r.status_code > 200 and flag > 0:
#             flag += 1
#             break
#         elif r.status_code > 200:
#             continue
#         else:
#             all_shape_urls.append(url)
            
# all_shape_urls


# all_geojsons = []

# import json

# for url in all_shape_urls:
#     resp = requests.get(url=url)
#     data = resp.json() # Check the JSON Response Content documentation below
#     all_geojsons.append(data)
    
# # Dump my new info into a pickle file.
# # now we have all of the geojson pages in a single pickle file (list object)

# with open('district_geojson.pkl', 'wb') as f:
#     pickle.dump(all_geojsons, f)

In [4]:
#all_geojsons = pickle.load( open( "datasets/district_geojson.pkl", "rb" ) )


In [5]:
# Now that I ran the above cell, I can just load the pickled data.
#all_geojsons = pickle.load( open( "datasets/district_geojson.pkl", "rb" ) )
#type(all_geojsons)

In [6]:
# Now build a df out of the geojsons.
# Each row will hold district and polygon info

from shapely.geometry import Polygon, Point, MultiPolygon, shape, GeometryCollection, asShape


# Let's start by making a dataframe that holds every district
    
my_district = all_geojsons[0]
my_district.keys()


# make a poly shape for each district
polygons = [asShape(x['geometry']) for x in all_geojsons]


district_names = [x['properties']['Code'] for x in all_geojsons]
df = pd.DataFrame(district_names, columns=['district'])

df['polygon'] = polygons

NameError: name 'all_geojsons' is not defined

In [ ]:
#!pip install geojson
df.head()


## MAKE A DISTRICT MAP

In [7]:
import folium

center= (39.8283, -98.5795, )

m = folium.Map(location=center, zoom_start=5)


for html in all_geojsons[:]:

    folium.GeoJson(
        html,
        name='geojson',
        control=True,
    ).add_to(m)


m.save('districts.html')


NameError: name 'all_geojsons' is not defined


I will be making this map in folium instead of plotly because of some problems I ran into with format of the geojson data.

## Build the dataset

In [8]:
import pickle


def get_district(point, dist_df):
    for i in range(len(dist_df)):
        poly = dist_df.iloc[i]['polygon']
        if poly.contains(point):
            return dist_df.iloc[i]['district']
        
        
def get_stores_by_dist(store_df, dist_df):
    found = store_df['points'].apply(get_district, args=[dist_df])
    return found.value_counts()


def return_count(district, wf_counts):
    try:
        return wf_counts[district]
    except:
        return 0

In [13]:
import pandas as pd
    
# Open the file from pickle to add to the df OR use it for rest of notebook
#df = pd.read_pickle('datasets/dist_store_df.pkl')

df = pickle.load( open( 'datasets/dist_store_df.pkl', "rb" ) )
    
    
    
# For each store...
# 1) import
# 2) get a list of discticts for each store location
# 3) use the list of district names to add a col to your df


# WHOLE FOODS 
# #1
# wf_df = pd.read_csv('datasets/wholefoods.csv', encoding='latin')
# # wf_df.to_csv('datasets/wholefoods.csv')  # fixing the encoding issue
# wf_df['longlat'] = list(zip(wf_df['long'], wf_df['lat']))
# wf_df['points'] = wf_df['longlat'].apply(make_point)

# #2
# wf_counts = get_stores_by_dist(wf_df, df) # get the district list

# #3
# df['WholeFoods'] = df.district.apply(return_count, args=[wf_counts])



# # TRACTOR SUPPLY
# ts_df = pd.read_csv('datasets/tractorsupply.csv')
# ts_df.info()
# ts_df = add_longlat(ts_df)
# ts_df.describe()
# #2
# ts_counts = get_stores_by_dist(ts_df, df) # get the district list

# #3
# df['TractorSupply'] = df.district.apply(return_count, args=[ts_counts])




# # STARBUCKS
# sb_df = pd.read_csv('datasets/starbucks.csv')
# sb_df = sb_df.rename(columns={'Longitude': "long", "Latitude":'lat'})
# sb_df = sb_df[sb_df['Country']=='US']
# sb_df = add_longlat(sb_df)

# #2
# sb_counts = get_stores_by_dist(sb_df, df) # get the district list

# #3
# df['Starbucks'] = df.district.apply(return_count, args=[sb_counts])



# ACADEMY SPORTS
# as_df = pd.read_csv('datasets/academy.csv')
# as_df = add_longlat(as_df)
# as_df.describe()

# #2
# as_counts = get_stores_by_dist(as_df, df) # get the district list

# #3
# df['AcademySports'] = df.district.apply(return_count, args=[as_counts])





# # DICK'S SPORTS
# ds_df = pd.read_csv('datasets/dicks.csv')
# ds_df = add_longlat(ds_df)
# ds_df.describe()

# #2
# ds_counts = get_stores_by_dist(ds_df, df) # get the district list

# #3
# df['DicksSports'] = df.district.apply(return_count, args=[ds_counts])



# # BASS PRO SHOPS
# bp_df = pd.read_csv('datasets/basspro.csv')
# bp_df = add_longlat(bp_df)
# bp_df.describe()

# #2
# bp_counts = get_stores_by_dist(bp_df, df) # get the district list

# #3
# df['BassPro'] = df.district.apply(return_count, args=[bp_counts])





# # DOLLAR TREE
# dt_df = pd.read_csv('datasets/dollartree.csv')
# dt_df = add_longlat(dt_df)
# dt_df.describe()

# #2
# dt_counts = get_stores_by_dist(dt_df, df) # get the district list

# #3
# df['DollarTree'] = df.district.apply(return_count, args=[dt_counts])






# # # GRAINGER STORE
# grg_df = pd.read_csv('datasets/grainger.csv')
# grg_df.info()
# grg_df = add_longlat(grg_df)  # this one had a bunch of garbage and extra commas
# grg_df.describe()

# #2
# grg_counts = get_stores_by_dist(grg_df, df) # get the district list

# #3
# df['Grainger'] = df.district.apply(return_count, args=[grg_counts])





# # CHICK FIL A
# cf_df = pd.read_csv('datasets/chickfila.csv', encoding='latin')
# cf_df.info()
# #cf_df.to_csv('datasets/chickfila.csv')
# #time.sleep(5)
# #cf_df = pd.read_csv('datasets/chickfila.csv', encoding='latin')
# cf_df = add_longlat(cf_df)  # format was wrong, had to resave as utf-8
# cf_df.describe()

# #2
# cf_counts = get_stores_by_dist(cf_df, df) # get the district list

# #3
# df['ChickFilA'] = df.district.apply(return_count, args=[cf_counts])





# # CRACKER BARREL
# # This one was pickled from scraping
# with open('datasets/crackerbarrel.pkl','rb') as f:
#     cb_df = pickle.load(f)

# cb_df = pd.DataFrame(cb_df)
# cb_df.columns = ['long', 'lat']
# cb_df = add_longlat(cb_df)
# cb_df.info()

# #2
# cb_counts = get_stores_by_dist(cb_df, df) # get the district list

# #3
# df['CrackerBarrel'] = df.district.apply(return_count, args=[cb_counts])




# # HARLEY DAVIDSON
# hd_df = pd.read_csv('datasets/harley.csv', encoding='latin')
# #dt_hd.to_csv('datasets/harley.csv')  # did this to fix encoding
# hd_df = add_longlat(hd_df)
# hd_df.describe()

# #2
# hd_counts = get_stores_by_dist(hd_df, df) # get the district list

# #3
# df['HarleyDavidson'] = df.district.apply(return_count, args=[hd_counts])




# # H&M
# # This one was pickled from scraping
# with open('datasets/hm.pkl','rb') as f:
#     hm_df = pickle.load(f)

# hm_df = pd.DataFrame(hm_df)
# hm_df.columns = ['long', 'lat']
# hm_df = add_longlat(hm_df)
# hm_df.info()

# #2
# hm_counts = get_stores_by_dist(hm_df, df) # get the district list

# #3
# df['HandM'] = df.district.apply(return_count, args=[hm_counts])




# # Hobby Lobby
# # This one was pickled from scraping
# with open('datasets/hobbylobby.pkl','rb') as f:
#     hl_df = pickle.load(f)

# hl_df = pd.DataFrame(hl_df)
# hl_df.columns = ['long', 'lat']
# hl_df = add_longlat(hl_df)
# hl_df.info()

# #2
# hl_counts = get_stores_by_dist(hl_df, df) # get the district list

# #3
# df['HobbyLobby'] = df.district.apply(return_count, args=[hl_counts])





# # LL Bean
# ll_df = pd.read_csv('datasets/llbean.csv')
# ll_df.info()
# ll_df = add_longlat(ll_df)  
# ll_df.describe()

# #2
# ll_counts = get_stores_by_dist(ll_df, df) # get the district list

# #3
# df['LLBean'] = df.district.apply(return_count, args=[ll_counts])




# # Pottery Barn
# pb_df = pd.read_csv('datasets/potterybarn.csv')
# pb_df.info()
# pb_df = add_longlat(pb_df)  
# pb_df.describe()

# #2
# pb_counts = get_stores_by_dist(pb_df, df) # get the district list

# #3
# df['PotteryBarn'] = df.district.apply(return_count, args=[pb_counts])



# # REI
# rei_df = pd.read_csv('datasets/rei.csv')
# rei_df.info()
# rei_df = add_longlat(rei_df)  
# rei_df.describe()

# #2
# rei_counts = get_stores_by_dist(rei_df, df) # get the district list

# #3
# df['REI'] = df.district.apply(return_count, args=[rei_counts])




# # Target
# tg_df = pd.read_csv('datasets/target.csv', encoding='latin')
# tg_df = tg_df.rename(columns={'Address.Longitude': "long", "Address.Latitude":'lat'})
# tg_df.info()
# #tg_df.to_csv('datasets/target.csv')  # did this to fix encoding
# tg_df = add_longlat(tg_df)  
# tg_df.describe()

# #2
# tg_counts = get_stores_by_dist(tg_df, df) # get the district list

# #3
# df['Target'] = df.district.apply(return_count, args=[tg_counts])





# # Trader Joe's
# tj_df = pd.read_csv('datasets/traderjoes.csv')
# tj_df.info()
# tj_df = add_longlat(tj_df)  
# tj_df.describe()

# #2
# tj_counts = get_stores_by_dist(tj_df, df) # get the district list

# #3
# df['TraderJoes'] = df.district.apply(return_count, args=[tj_counts])




# # WalMart. 
# # This one was pickled from scraping
# with open('datasets/walmart.pkl','rb') as f:
#     wal_df = pickle.load(f)

# wal_df = pd.DataFrame(wal_df)

# wal_df
# wal_df.columns = ['long', 'lat']
# wal_df = add_longlat(wal_df)
# wal_df.info()

# #2
# wal_counts = get_stores_by_dist(wal_df, df) # get the district list

# #3
# df['Walmart'] = df.district.apply(return_count, args=[wal_counts])





# # COSTCO
# cc_df = pd.read_csv('datasets/costco.csv', encoding='latin')
# #cc_df = cc_df.drop(columns=['Unnamed: 0'])
# #cc_df.to_csv('datasets/costco.csv')  # did this to fix encoding
# cc_df.info()
# cc_df = add_longlat(cc_df)  
# cc_df.info()

# #2
# cc_counts = get_stores_by_dist(cc_df, df) # get the district list

# #3
# df['Costco'] = df.district.apply(return_count, args=[cc_counts])








# UNCOMMENT THIS LINE TO SAVE THE DATA AGAIN
# df.to_pickle("datasets/dist_store_df.pkl")

TypeError: __init__() missing 1 required positional argument: 'context'

In [10]:
df.describe()


NameError: name 'df' is not defined

## 2018 Midterm Election Data

The election data is taken from https://electionlab.mit.edu/data.
The data has results for every election, both state and national.  It includes the vote total for each candidate and the candidate's party.

We will use the fips code for each county to act as an index for the more than 3000 counties contained in the dataset.  Fips codes have a two digit state id followed by a three digit county id code within the state.  States are listed in alphabetical order.  The dataset also contains information on territories (Guam, NMI, PR, etc.) but that will be filtered out to focus on the contiguous United States and Hawaii.  The burroughs of Alaska are unfortunately not part of this dataset, although after looking at the frequency of stores in Alaska, it might  not be useful to add to the model as many of these stores are not present or sparse in our 49th state.  District of Columbia is unfortunately not included.  Adding the Alaskan burroughs and DC would be a recommendation for further study.

In [ ]:
# read in election results from csv
df = pd.read_csv('datasets/distric_overall_2018.csv', encoding='latin') 
df.to_csv('datasets/distric_overall_2018.csv')

# Eliminates territories (past Wyoming last alphabetically)
# df = df[df['state_fips'].apply(int) < 57]
# df['state_fips'].unique()

### Filter the election data 
I chose to go with only the US House races.  
Senators tend to have long term support statewide (in some states) and might influence the model.

Further investigation: you could look at all races cumulative to determine red or blue counties.
Many local elections are decided cross party, unlike congressional elections.

In [ ]:
# remove all lines that are not republican or democrat
df.party.unique() # two party data only, there are lots of them 
df = df[(df['party']=='republican') | (df['party']=='democrat')]  # df2 is two party only
df.party.value_counts()

In [ ]:
# I would like to see if model perfoms better on national vs. local elections, so I can try both here.  
# Might show the partisan lean better than state and local
# Three imported lists exist (all_offices, major_offices, house and congress), see myfunctions module


# # Put in the data you wish to include in the model.  You could use congress, major, or all (I chose house)
df2 = df[df['office'].isin(all_offices)]  # all_offices is a list from import

df_iowa = df[df['state_fips'] == 19]

# This is a little fix for Iowa which is missing one congressional race if you want to look at house.
# Data won't be perfect, but close
df2 = pd.concat([df2, df_iowa])


# eliminate unnecessary columns
df2 = df2[['state', 'county', 'state_fips', 'party', 'candidatevotes', 'totalvotes', 'office']]

df2.info()


In [ ]:
# We decided to just count up all of the votes for each party in every election.
# If we chose to include local elections or major elections, this would
# weight congressional elections the same as local, but is aimed at getting a sense of how much
# the county leans red or blue without individual politicians affecting the categorization
df2_grouped = df2.groupby(by=['state_fips', 'county', 'party']).sum().reset_index()

# This drops out territories by chopping off everything after Wyoming
df2_grouped = df2_grouped[df2_grouped['state_fips'].apply(int) < 57]
df2_grouped.head()

# we are left with vote totals for GOP and Dem for each county

### Cleaning up the county data
County naming is not standardized. There were lots of exceptions in the more than 3000 counties that had to be addressed. The data in the us_county dataset was not named the same as election data. Each .replace was a manual change to get the two datasets to conform. The election data did not have FIPS data initially included. That is unfortunate.

In [ ]:
# Make a new col named fips which is initially filled with zeroes
df2_grouped_fips = df2_grouped.copy()
df2_grouped_fips['fips'] = pd.Series([0 for x in range(len(df2_grouped_fips.index))])  


# The code below ensures that county names match exactly so we can populate the fips column
# this will make fips a key used for election, county, and population datasets
us_county['NAME'] = us_county['NAME'].apply(lambda x: x.upper())
us_county['NAME'] = us_county['NAME'].apply(lambda x: x.replace('.', '')
                                                              .replace("''", '')
                                                              .replace("DE WITT", 'DEWITT')
                                                              .replace('LA SALLE', 'LASALLE')
                                                              .replace("DE KALB", 'DEKALB')
                                                              .replace(" CITY", '')
                                                              .strip())




df2_grouped_fips['county'] = df2_grouped_fips['county'].apply(lambda x: x.upper())
df2_grouped_fips['county'] = df2_grouped_fips['county'].apply(lambda x: x.replace('COUNTY', '')
                                                                          .replace('.', '')
                                                                          .replace(' CITY', '')
                                                                          .replace('MEEER', 'MEEKER')
                                                                          .replace('JODAVIESS', 'JO DAVIESS')
                                                                          .replace('&', 'AND')
                                                                          .replace('DE WITT', 'DEWITT')
                                                                          .replace('DE KALB', 'DEKALB')
                                                                          .replace('LAC QUI PARTE', 'LAC QUI PARLE')
                                                                          .replace('OGLALA LAKOTA', 'OGLALA')
                                                                          .replace('CHENAGO', 'CHENANGO')
                                                                          .replace('LA SALLE', 'LASALLE')
                                                                          .replace('DONA ANA', 'DOÑA ANA')
                                                                          .strip())

# These are county names we dropped
# They inclued UOCAVA (overseas votes), Oglala Lakota and some other NON-COUNTY data
df2_grouped_fips = df2_grouped_fips.loc[~df2_grouped_fips['county'].isin(['STATE TOTALS', 
                                                                            'STATE UOCAVA', 
                                                                            'TOTAL VOTES BY CANDIDATE',
                                                                            'TOTAL VOTES BY PARTY',
                                                                            'KANSAS',
                                                                            'OGLALA',
                                                                            'FEDERAL PRECINCT',
                                                                          
                                                                        ])]

# Force numperic values 
us_county.info()
df2_grouped_fips.info()
us_county["STATE"] = pd.to_numeric(us_county["STATE"])
us_county["fips"] = pd.to_numeric(us_county["fips"])




In [ ]:
# I used this loop to go back and correct the county names between the two datasets
errors = 0  # track the dumped counties

for i in range(len(df2_grouped_fips)):
    state = int(df2_grouped_fips.iloc[i, :]['state_fips'])  # get state number (1 to 57 numeric)
    county = df2_grouped_fips.iloc[i, :]['county'].strip() # get county name
    #print(us_county_df.loc[us_county_df['NAME'] == county].iloc[-1][-1])
    try:
        # try to associate add the approprate fips for each county
        fip = us_county.loc[(us_county['NAME']==county) & (us_county['STATE']==state)].iloc[-1][-1]
        df2_grouped_fips.iloc[i, -1] = fip  # BE CAREFUL HERE
    except:
        # if it didn't work, print it out for troubleshooting
        print(county, state, i)
        errors +=1
        

print(errors)

In [ ]:
df_final = df2_grouped_fips.copy().reset_index()  # failed at index 414, index was missing for agg(idxmax) (Fixed)


# get only highest (DEM or GOP)
# This eliminates the 'loser' of each county (this is aggregate votes, not individual elections)
df_final = df_final.iloc[df_final.groupby('fips')['candidatevotes'].agg(pd.Series.idxmax)]
#df2_final = df2_final.iloc[df2_final.groupby('fips')['candidatevotes'].idxmax().values.ravel()]

df_final.describe()


# FINAL TALLY
# US HAS 3141 total counties.

# Missing counties
# DC has no counties.  Not sure how to handle that
# MISSING OGLALA LAKOTA county (Native American lands)
# MISSING the 19 ALASKA buroughs data 
# MAY BE MISSING MORE COUNTY EQUIVALENTS

# Data is missing for Iowa US Congressional race, must use all offices instead.  bummer

In [ ]:
df2_final = df_final.copy()



# MAKE MY FIPS COMPATIBLE WITH THE GEODATA
df2_final['fips'] = df2_final['fips'].apply(lambda x: "{:05}".format(x))

df2_final['blue'] = df2_final['party'].apply(lambda x: 0 if (x=='republican') else 1)

In [ ]:
df2_final.head()


All datasets are stored in the /datasets folder which contains:
- academy.csv			
- county_boundaries_new.json	
- grainger.csv			
- rei.csv				
- walmart.pkl
- basspro.csv			
- county_fips_master.csv		
- harley.csv			
- starbucks.csv			
- wholefoods.csv
- chickfila.csv			
- crackerbarrel.pkl		
- hm.pkl				
- store_df.pkl
- counties_fixed.geojson		
- dicks.csv			
- hobbylobby.pkl			
- target.csv
- county_2018.csv			
- district_geojson.pkl		
- llbean.csv			
- tractorsupply.csv
- county_boundaries.json		
- dollartree.csv			
- potterybarn.csv			
- traderjoes.csv

In [ ]:


# Add in a chain stores
# COMMENTED OUT CODE TO IMPORT STORES.  TRYING TO DO IT ONE AT A TIME
# Each dataset was a little bit different, so I decided not to automate with functions

# # WHOLE FOODS 
# wf_df = pd.read_csv('datasets/wholefoods.csv', encoding='latin')
# # wf_df.to_csv('datasets/wholefoods.csv')  # fixing the encoding issue
# wf_df['longlat'] = list(zip(wf_df['long'], wf_df['lat']))
# wf_df['points'] = wf_df['longlat'].apply(make_point)


# # TRACTOR SUPPLY
# ts_df = pd.read_csv('datasets/tractorsupply.csv')
# ts_df.info()
# ts_df = add_longlat(ts_df)
# ts_df.describe()


# # STARBUCKS
# sb_df = pd.read_csv('datasets/starbucks.csv')
# sb_df = sb_df.rename(columns={'Longitude': "long", "Latitude":'lat'})
# sb_df = sb_df[sb_df['Country']=='US']
# sb_df = add_longlat(sb_df)


# # ACADEMY SPORTS
# as_df = pd.read_csv('datasets/academy.csv')
# as_df = add_longlat(as_df)
# as_df.describe()

# # DICK'S SPORTS
# ds_df = pd.read_csv('datasets/dicks.csv')
# ds_df = add_longlat(ds_df)
# ds_df.describe()


# # BASS PRO SHOPS
# bp_df = pd.read_csv('datasets/basspro.csv')
# bp_df = add_longlat(bp_df)
# bp_df.describe()

# # DOLLAR TREE STORE
# dt_df = pd.read_csv('datasets/dollartree.csv')
# dt_df = add_longlat(dt_df)
# dt_df.describe()

# # GRAINGER STORE
# grg_df = pd.read_csv('datasets/grainger.csv')
# grg_df.info()
# grg_df = add_longlat(grg_df)  # this one had a bunch of garbage and extra commas
# grg_df.describe()


# # CHICK FIL A
# cf_df = pd.read_csv('datasets/chickfila.csv', encoding='latin')
# cf_df.info()
# #cf_df.to_csv('datasets/chickfila.csv')
# #time.sleep(5)
# #cf_df = pd.read_csv('datasets/chickfila.csv', encoding='latin')
# cf_df = add_longlat(cf_df)  # format was wrong, had to resave as utf-8
# cf_df.describe()


# # CRACKER BARREL
# # This one was pickled from scraping
# with open('datasets/crackerbarrel.pkl','rb') as f:
#     cb_df = pickle.load(f)

# cb_df = pd.DataFrame(cb_df)
# cb_df.columns = ['long', 'lat']
# cb_df = add_longlat(cb_df)
# cb_df.info()


# # HARLEY DAVIDSON
# hd_df = pd.read_csv('datasets/harley.csv', encoding='latin')
# #dt_hd.to_csv('datasets/harley.csv')  # did this to fix encoding
# hd_df = add_longlat(hd_df)
# hd_df.describe()


# # H&M
# # This one was pickled from scraping
# with open('datasets/hm.pkl','rb') as f:
#     hm_df = pickle.load(f)

# hm_df = pd.DataFrame(hm_df)
# hm_df.columns = ['long', 'lat']
# hm_df = add_longlat(hm_df)
# hm_df.info()


# # Hobby Lobby
# # This one was pickled from scraping
# with open('datasets/hobbylobby.pkl','rb') as f:
#     hl_df = pickle.load(f)

# hl_df = pd.DataFrame(hl_df)
# hl_df.columns = ['long', 'lat']
# hl_df = add_longlat(hl_df)
# hl_df.info()

# # LL Bean
# ll_df = pd.read_csv('datasets/llbean.csv')
# ll_df.info()
# ll_df = add_longlat(ll_df)  
# ll_df.describe()


# # Pottery Barn
# pb_df = pd.read_csv('datasets/potterybarn.csv')
# pb_df.info()
# pb_df = add_longlat(pb_df)  
# pb_df.describe()


# # REI
# rei_df = pd.read_csv('datasets/rei.csv')
# rei_df.info()
# rei_df = add_longlat(rei_df)  
# rei_df.describe()


# # Target
# tg_df = pd.read_csv('datasets/target.csv', encoding='latin')
# tg_df = tg_df.rename(columns={'Address.Longitude': "long", "Address.Latitude":'lat'})
# tg_df.info()
# #tg_df.to_csv('datasets/target.csv')  # did this to fix encoding
# tg_df = add_longlat(tg_df)  
# tg_df.describe()


# # Trader Joe's
# tj_df = pd.read_csv('datasets/traderjoes.csv')
# tj_df.info()
# tj_df = add_longlat(tj_df)  
# tj_df.describe()

# # WalMart. 
# # This one was pickled from scraping
# with open('datasets/walmart.pkl','rb') as f:
#     wal_df = pickle.load(f)

# wal_df = pd.DataFrame(wal_df)

# wal_df
# wal_df.columns = ['long', 'lat']
# wal_df = add_longlat(wal_df)
# wal_df.info()


# COSTCO
cc_df = pd.read_csv('datasets/costco.csv', encoding='latin')
#cc_df = cc_df.drop(columns=['Unnamed: 0'])
#cc_df.to_csv('datasets/costco.csv')  # did this to fix encoding
cc_df.info()
cc_df = add_longlat(cc_df)  
cc_df.info()

In [ ]:
def get_county(point, us_county):
    for i in range(len(us_county)):
        poly = us_county.iloc[i]['geometry']
        if poly.contains(point):
            return us_county.iloc[i]['fips']
        
def get_stores_by_county(store_df, us_county):
    found = store_df['points'].apply(get_county, args=[us_county])
    return found.value_counts()


# <<<<<<<<<<<<HIGH RESOURCE CODE ALERT>>>>>>>>>>>>
#wf_counts = get_stores_by_county(wf_df, us_county)  # WHOLE FOODS
#ts_counts = get_stores_by_county(ts_df, us_county)  # TRACTOR SUPPLY STORE
#sb_counts = get_stores_by_county(sb_df, us_county)  # STARBUCKS COFFEE 
#as_counts = get_stores_by_county(as_df, us_county)  # ACADEMY SPORTS  
#ds_counts = get_stores_by_county(ds_df, us_county)  # DICK'S SPORTS
#bp_counts = get_stores_by_county(bp_df, us_county)  # BASS PRO
#dt_counts = get_stores_by_county(dt_df, us_county)  # DOLLAR TREE
#grg_counts = get_stores_by_county(grg_df, us_county)  # GRAINGER
#cf_counts = get_stores_by_county(cf_df, us_county)  # CHICK-FIL-A
#cb_counts = get_stores_by_county(cb_df, us_county)  # CRACKER BARREL
#hd_counts = get_stores_by_county(hd_df, us_county)  # HARLEY DAVIDSON
#hm_counts = get_stores_by_county(hm_df, us_county)  # H and M
#hl_counts = get_stores_by_county(hl_df, us_county)  # HOBBY LOBBY
#ll_counts = get_stores_by_county(ll_df, us_county)  # LL BEAN
#pb_counts = get_stores_by_county(pb_df, us_county)   # POTTERY BARN
#rei_counts = get_stores_by_county(rei_df, us_county)   # REI
#tg_counts = get_stores_by_county(tg_df, us_county)   # TARGET
#tj_counts = get_stores_by_county(tj_df, us_county)   # TRADER JOE'S
#wal_counts = get_stores_by_county(wal_df, us_county)   # WALMART
#cc_counts = get_stores_by_county(cc_df, us_county)   # COSTCO

########################

In [ ]:
# All of the resource intensive code is already run and has been pickled

#  Open the pickle jar
with open('datasets/store_df.pkl','rb') as f:
    df_stores = pickle.load(f)


# Implement this as .apply function later

# def is_blue(fips, df):
#     try:
#         blue = df.loc[df['fips']==fips]
#         return blue.value
#     except Exception as e:
#         print(fips, e)

# something like this
# df_stores['fips'].apply(is_blue, args=(df2_final,)) 
# df_stores['fips'].apply(is_blue, df2_final)


# For now
blue = []

for i in range(len(df_stores)):
    f = df_stores.iloc[i,6]  # hardcoded iloc to get fips code
    b = df2_final[df2_final['fips']==str(f)]['blue']
    blue.append(int(b))

df_stores['blue'] = blue
df_stores.describe()

In [ ]:
import pickle

def return_count(fips, wf_counts):
    try:
        return wf_counts[int(fips)]
    except:
        return 0

    
# These were added one at a time
# df_stores['WholeFoods'] = df_stores.fips.apply(return_count, args=[wf_counts])
# df_stores['TractorSupply'] = df_stores.fips.apply(return_count, args=[ts_counts])
# df_stores['Starbucks'] = df_stores.fips.apply(return_count, args=[sb_counts])
# df_stores['AcademySports'] = df_stores.fips.apply(return_count, args=[as_counts])
# df_stores['DicksSports'] = df_stores.fips.apply(return_count, args=[ds_counts])
# df_stores['BassPro'] = df_stores.fips.apply(return_count, args=[bp_counts])
# df_stores['DollarTree'] = df_stores.fips.apply(return_count, args=[dt_counts])
# df_stores['Grainger'] = df_stores.fips.apply(return_count, args=[grg_counts])
# df_stores['ChickFila'] = df_stores.fips.apply(return_count, args=[cf_counts])
# df_stores['CrackerBarrel'] = df_stores.fips.apply(return_count, args=[cb_counts])
# df_stores['HarleyDavidson'] = df_stores.fips.apply(return_count, args=[hd_counts])
# df_stores['HM'] = df_stores.fips.apply(return_count, args=[hm_counts])
# df_stores['HobbyLobby'] = df_stores.fips.apply(return_count, args=[hl_counts])
#df_stores['LLBean'] = df_stores.fips.apply(return_count, args=[ll_counts])
#df_stores['PotteryBarn'] = df_stores.fips.apply(return_count, args=[pb_counts])
#df_stores['REI'] = df_stores.fips.apply(return_count, args=[rei_counts])
#df_stores['Target'] = df_stores.fips.apply(return_count, args=[tg_counts])
#df_stores['TraderJoes'] = df_stores.fips.apply(return_count, args=[tj_counts])
#df_stores['Walmart'] = df_stores.fips.apply(return_count, args=[wal_counts])
#df_stores['Costco'] = df_stores.fips.apply(return_count, args=[cc_counts])


# UNCOMMENT THIS LINE TO SAVE THE DATA AGAIN
#df_stores.to_pickle("datasets/store_df.pkl")

In [ ]:

# quick imported geojson file for plotting
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    us_county_fix = json.load(response)

# EACH LINE MAKES A STORE CHOROPLETH MAP FOR US
# Each also is 'tuned' to best display store density

#make_choro(df_stores1, us_county_fix, 'WholeFoods', ['lightgray', 'green'], [0, 4])
#make_choro(df_stores2, us_county_fix, 'TractorSupply', ['lightgray', 'red'], [0, 4])
#make_choro(df_stores5, us_county_fix, 'Starbucks', ['white', 'green'], [0, 50])
#make_choro(df_stores5, us_county_fix, 'AcademySports', ['white', 'blue'], [0, 2])
#make_choro(df_stores5, us_county_fix, 'DicksSports', ['white', 'orange'], [0, 2])
#make_choro(df_stores7, us_county_fix, 'BassPro', ['lightyellow', 'red'], [0, 2])
#make_choro(df_stores7, us_county_fix, 'DollarTree', ['white', 'green'], [0, 50])
#make_choro(df_stores, us_county_fix, 'Grainger', ['lightgray', 'red'], [0, 3])
#make_choro(df_stores, us_county_fix, 'ChickFila', ['lightgray', 'red'], [0, 20])
#make_choro(df_stores, us_county_fix, 'CrackerBarrel', ['lightgray', 'gold'], [0, 4])
#make_choro(df_stores, us_county_fix, 'HarleyDavidson', ['lightgray', 'orange'], [0, 2])
#make_choro(df_stores, us_county_fix, 'HM', ['white', 'red'], [0, 2])
#make_choro(df_stores, us_county_fix, 'HobbyLobby', ['white', 'orange'], [0, 3])
#make_choro(df_stores, us_county_fix, 'LLBean', ['white', 'darkgreen'], [0, 1])
#make_choro(df_stores, us_county_fix, 'PotteryBarn', ['white', 'blue'], [0, 1])
#make_choro(df_stores, us_county_fix, 'REI', ['white', 'darkgreen'], [0, 1])
#make_choro(df_stores, us_county_fix, 'Target', ['white', 'red'], [0, 10])
#make_choro(df_stores, us_county_fix, 'TraderJoes', ['white', 'red'], [0, 5])
#make_choro(df_stores, us_county_fix, 'Costco', ['white', 'blue'], [0, 1])

#make_choro(df2_final, us_county_fix, 'blue', ['red', 'blue'], [0, 1])
df_stores.describe()


In [ ]:
# To complete my df_store and make it final, I want to do a couple experiments
# I want to simply add a population col
# I would like to make a second model using stores/person for each column

# Add population data
pop_df = pd.read_csv('datasets/county_pop.csv', encoding='latin')
#pop_df.to_csv('datasets/county_pop.csv')

# pop_df.info()
# for col in pop_df.columns:
#     print(col)
    
# Keep STATE, COUNTY, CTYNAME, POPESTIMATE2018, 
pop_df = pop_df[['STATE', 'COUNTY', 'CTYNAME', 'POPESTIMATE2018']]
pop_df.head()
fips_list = []

for i in range(len(pop_df)):
    fips = "{:02}{:03}".format(pop_df.iloc[i]['STATE'], pop_df.iloc[i]['COUNTY'])
    fips_list.append(fips)
    
pop_df['fips'] = pd.Series(fips_list)

pop_df.head()

In [ ]:
df_stores.head()

In [ ]:
# Add population and area columns to my final dataset
# Helper functions in the import
def get_area(fips, us_county, area_index):
    try:
        area = us_county[us_county['fips']==fips].iloc[:, area_index]
        return float(area)
    except Exception as e:
        print(fips, e)

        

us_county['fips'] = us_county['fips'].apply(lambda x: "{:05}".format(int(x)))
area_index = list(us_county.columns).index('CENSUSAREA')




df_stores['population'] = df_stores.fips.apply(get_pop, args=[pop_df])
df_stores['area'] = df_stores.fips.apply(get_area, args=[us_county, area_index])

In [ ]:
# ONE EXCEPTION THROWN....
# the population of bedford city county va (fips 51515) is 6181.  Let's fill it in manually
df_stores.loc[df_stores['fips']=='51515']
df_stores.loc[5422,'population'] = 6181
df_stores.loc[df_stores['fips']=='51515']  # fixed

In [ ]:
# Looks good, let's pickle it and start our ML project

# UNCOMMENT THE FILE YOU WANT TO PICKLE TO
# may want to automate this later with a variable depending on offices chosen

#df_stores.to_pickle("datasets/house_df.pkl")
df_stores.to_pickle('datasets/all_df.pkl')
#df_stores.to_pickle('datasets/major_df.pkl')

In [ ]:
# little extra to fix the df before finally sending it to pickle
# This code was run once only to fix the dataset after loading the store data

# import pickle
# import pandas as pd

# # import data

# with open('datasets/final_df.pkl', 'rb') as f:
#     df = pickle.load(f)
    
# #had an extra column and used this to fix it
# df = df.drop(df.columns[-1], axis=1)


# df.party = df.party.apply(lambda x: x.title())
# df.county = df.county.apply(lambda x: x.title())
# # df = df.drop(columns=['totalvotes'], axis=1)

# df.head()
# #df.to_pickle('datasets/final_df.pkl')
# df.to_pickle('datasets/house_df.pkl')